# LSTM STOCK FORECASTING

In [ ]:
pip install yfinance --upgrade --no-cache-dir

# BBCA.JK STOCK

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import datetime as dt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.metrics import mean_squared_error

#untuk load data saham (ticker)
company = str(
    input(
        "Masukan Ticker saham yang ingin dianalisis program, sebagai contoh (BBCA.JK): "
    ))  #'UNVR.JK'#atau SIDO.JK; BBCA.JK

#Penentuan penanggalan mulai dan berakhir
start = dt.datetime(2012, 1, 1)
end = dt.date.today()

data = yf.download(company, start, end)
data = data.dropna()
data_close = data["Close"]

error_tolenransi = 0.04  #0.031983333

status_punya_saham = str(
    input(
        "Apakah anda sedang 'tidak' memiliki saham atau 'ingin membeli' saham baru? Jawab dengan ya/tidak: "
    ))
if status_punya_saham == "ya":
    punya_saham = 0
else:
    punya_saham = 1

kasus_pajak_beli = str(
    input(
        "Apakah anda menginginkan 'ongkos beli' (pajak beli) berbeda (bawaan adalah 0.0019), jawab dengan ya/tidak: "
    ))
if kasus_pajak_beli == "ya":
    pajak_beli = float(
        input(
            "Masukan besarnya 'ongkos beli' (pajak beli) baru dengan decimal: "
        ))
else:
    pajak_beli = 0.0019

kasus_pajak_jual = str(
    input(
        "Apakah anda menginginkan 'ongkos jual' (pajak jual) berbeda (bawaan adalah 0.0039), jawab dengan ya/tidak: "
    ))
if kasus_pajak_jual == "ya":
    pajak_jual = float(
        input(
            "Masukan besarnya 'ongkos jual' (pajak jual) baru dengan decimal: "
        ))
else:
    pajak_jual = 0.0039

list_banyak_error_toleransi = [0]

lompatan = 1

harga_beli_saham = data_close.iloc[
    len(data_close.index) -
    1] + pajak_beli * data_close.iloc[len(data_close.index) - 1]

while lompatan < 11:
    end = dt.date.today() + dt.timedelta(
        days=lompatan)  #Days bisa digant bersesuaian

    #mempersiapkan datanya
    scaler = MinMaxScaler(feature_range=(0, 1))  #dia normalisasi data
    scaled_data = scaler.fit_transform(data_close.values.reshape(-1, 1))

    #Prediksi forecasting berdasarkan 60 hari sebelumnya????
    prediction_days = round(
        4 / 100 * data_close.size)  #ini banyak hari untuk testing data
    #ini tadinya 60 hari

    #list untuk hasil training
    x_train = []  #kayaknya ini
    y_train = []

    #kasus ini kenapa tidak pengambilan acak, karena kita mau belajar dari historisnya, buat prediksi si 60 hari itu nanti
    for x in range(
            prediction_days, len(scaled_data)
    ):  #ini x dari 120 sampai akhir ujung hasil minmax data close (2000an)
        x_train.append(scaled_data[x - prediction_days:x, 0])
        y_train.append(scaled_data[x, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    #membuat model
    model = Sequential()

    model.add(
        LSTM(units=50,
             return_sequences=True,
             input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(0.2))

    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(units=50))
    model.add(Dropout(0.2))

    model.add(Dense(units=1))  #prediction untuk next closenya

    #============================================================
    model.compile(optimizer="adam", loss="mean_squared_error")
    model.fit(x_train, y_train, epochs=1,
              batch_size=32)  #nanti epochs balikin ke 25

    #test model akurasi dari data yang dipunya
    #load test datanya
    #===================================================== ada catatan di bawah:
    test_data = data_close  #yf.download(company, start,end)
    test_data = test_data.dropna()
    actual_prices = data_close.values

    #bagian concatnya keknya rada-rada...
    total_dataset = pd.concat((data_close, test_data), axis=0)

    #ada unsur kesengajaan di total_dataset, concatnya 2 hal yang sama...
    #beigtu juga pada model_inputs
    #dibagian total_dataset, langsung aja pake len(test_data)

    model_inputs = total_dataset[
        len(total_dataset) - len(test_data) -
        prediction_days:].values  #yang dipake buat belajar?
    model_inputs = model_inputs.reshape(-1, 1)
    model_inputs = scaler.transform(model_inputs)
    #=====================================================

    #hasil dari prediction dari test data
    x_test = []

    for x in range(prediction_days, len(model_inputs)):
        x_test.append(model_inputs[x - prediction_days:x, 0])

    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    predicted_prices = model.predict(x_test)
    predicted_prices = scaler.inverse_transform(predicted_prices)

    rmse = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
    #Memprediksi saham sekian hari kedepan
    real_data = [
        model_inputs[len(model_inputs) - prediction_days:len(model_inputs + 1),
                     0]
    ]
    real_data = np.array(real_data)
    real_data = np.reshape(real_data,
                           (real_data.shape[0], real_data.shape[1], 1))

    prediction = model.predict(real_data)
    prediction = scaler.inverse_transform(prediction)
    print("Maka hasil forecasting saham untuk ke\t",dt.date.today()+dt.timedelta(days=lompatan),\
          "\tadalah:","\n" f"ini adalah hasil prediksinya {prediction}",\
          "\nDimana RMSEnya adalah:", rmse, "\n")

    data_close.loc[len(data_close.index)] = prediction  #append hasil predict

    print(data_close.iloc[len(data_close.index) - 2],
          f"harga saat hari ke: {lompatan-1}\n")
    print(data_close.iloc[len(data_close.index) - 1],
          f"harga saat hari ke: {lompatan}\n")

    keadaan_saham_error = abs(data_close.iloc[len(data_close.index) - 2] -
                              data_close.iloc[len(data_close.index) - 1]
                              ) / data_close.iloc[len(data_close.index) - 2]
    #harga_beli_saham=data_close.iloc[len(data_close.index)-1] + pajak_beli * data_close.iloc[len(data_close.index)-1]

    if punya_saham == 0:
        if data_close.iloc[len(data_close.index) -
                           2] < data_close.iloc[len(data_close.index) - 1]:
            punya_saham = 1
            harga_beli_saham = data_close.iloc[
                len(data_close.index) -
                2] + pajak_beli * data_close.iloc[len(data_close.index) - 2]
            print(
                f"Beli saham pada hari ke {lompatan-1}, karena besok akan naik\n"
            )
            print(
                f"Harga saham yang dibeli adalah {data_close.iloc[len(data_close.index)-2]}\n"
            )

    if punya_saham == 1:
        if lompatan == 1:
            if data_close.iloc[len(data_close.index) -
                               2] > data_close.iloc[len(data_close.index) - 1]:
                #if keadaan_saham_error > error_tolenransi:
                harga_jual_saham = data_close.iloc[
                    len(data_close.index) -
                    1] - pajak_jual * data_close.iloc[len(data_close.index) -
                                                      1]
                if harga_beli_saham < harga_jual_saham:
                    print("Saham harus dijual saat hari: ", lompatan - 1,
                          "karena besok akan jatuh \n")
                    print(
                        "Keuntungan yang akan didapat persatuan sahamnya sebesar: ",
                        harga_jual_saham - harga_beli_saham)
                    lompatan = lompatan + 100

                else:
                    print("Saham merugi, tetapi saham tidak bisa dijual saat hari: ", lompatan-1, """karena tidak didapat keuntungan dari
          selisih harga jual saham dan harga beli saham setelah pajaknya 
          dengan nilai: \n""", f"ini adalah nilai harga jual saham setelah pajak: {harga_jual_saham}\n",\
                    f"ini adalah nilai harga beli sahamnya setelah pajak: {harga_beli_saham}")

            else:
                #list_banyak_error_toleransi=[0]
                print(
                    "Saham dalam keadaan baik dan meningkat untuk hari prediksi ke: ",
                    lompatan, "\n")

        else:
            if data_close.iloc[len(data_close.index) -
                               2] > data_close.iloc[len(data_close.index) - 1]:
                #if keadaan_saham_error > error_tolenransi:
                harga_jual_saham = data_close.iloc[
                    len(data_close.index) -
                    2] - pajak_jual * data_close.iloc[len(data_close.index) -
                                                      2]
                if harga_beli_saham < harga_jual_saham:
                    print("Saham harus dijual saat hari: ", lompatan - 1,
                          "karena besok akan jatuh \n")
                    print(
                        "Keuntungan yang akan didapat persatuan sahamnya sebesar: ",
                        harga_jual_saham - harga_beli_saham)
                    lompatan = lompatan + 100

                else:
                    print("Saham merugi, tetapi saham tidak bisa dijual saat hari: ", lompatan-1, """karena tidak didapat keuntungan dari
          selisih harga jual saham dan harga beli saham setelah pajaknya 
          dengan nilai: \n""", f"ini adalah nilai harga jual saham setelah pajak: {harga_jual_saham}\n",\
                    f"ini adalah nilai harga beli sahamnya setelah pajak: {harga_beli_saham}")

                    #else:
                    #print("Keadaan sedang merugi, tetapi masih dibawah batas toleransi pada hari prediksi ke: ", lompatan, "\n")
                    #list_banyak_error_toleransi.append(1)
            else:
                #list_banyak_error_toleransi=[0]
                print(
                    "Saham dalam keadaan baik dan meningkat untuk hari prediksi ke: ",
                    lompatan, "\n")

    #if sum(list_banyak_error_toleransi)>=2:
    #print("kejadian saham merugi sudah 2 kali, maka program stop dan disarankan untuk menjual pada hari ke: ",lompatan-2)
    #lompatan=100

    lompatan = lompatan + 1
predicted_prices_plot = predicted_prices[:len(predicted_prices) -
                                         (lompatan - 100) + 2]
for i in actual_prices[len(actual_prices) - (lompatan - 100) + 2:]:
    predicted_prices_plot = np.append(predicted_prices_plot, i, axis=0)

#Plotting test prediksinya dan eksaknya
plt.figure(figsize=(15, 7))
plt.grid(True)
plt.plot(actual_prices[:len(actual_prices) - (lompatan - 100) + 2],
         color="green",
         label=f"Actual {company} Price")
plt.plot(predicted_prices_plot,
         color="pink",
         label=f"Predicted {company} Price")
plt.title(f"{company} Share Price")
plt.xlabel("Time")
plt.ylabel(f"{company} Share Price")
plt.legend()
plt.show()